In [2]:
import random

bloquept = 8
mediobloquept = int(bloquept/2)
tamaniollave = 8
tamaniosubllave = 6
cantSbox = int(tamaniosubllave/6)
salidaSbox = int(mediobloquept/cantSbox)

In [3]:
def generatePermTab(n):
	table=[*range(1,n+1,1)]
	for i in range(len(table)-1):
		random_index = random.randint(0, (len(table)-1))
		temp = table.pop(random_index)
		table.append(temp)
	return table
	
def generateExpTab(mediobloquept, tamaniosubllave):
	values= [*range(1,mediobloquept+1, 1)]
	exp=[]
	for i in range(tamaniosubllave):
		exp.append(values[random.randint(0,(len(values)-1))])
	return exp

def pt2hex(s):
	hex_s = s.encode('utf-8')
	hex_s = hex_s.hex()
	return hex_s.upper()

# Hexadecimal to binary conversion


def hex2bin(s):
	mp = {'0': "0000",
		'1': "0001",
		'2': "0010",
		'3': "0011",
		'4': "0100",
		'5': "0101",
		'6': "0110",
		'7': "0111",
		'8': "1000",
		'9': "1001",
		'A': "1010",
		'B': "1011",
		'C': "1100",
		'D': "1101",
		'E': "1110",
		'F': "1111"}
	bin = ""
	for i in range(len(s)):
		bin = bin + mp[s[i]]
	return bin

# Binary to hexadecimal conversion


def bin2hex(s):
	mp = {"0000": '0',
		"0001": '1',
		"0010": '2',
		"0011": '3',
		"0100": '4',
		"0101": '5',
		"0110": '6',
		"0111": '7',
		"1000": '8',
		"1001": '9',
		"1010": 'A',
		"1011": 'B',
		"1100": 'C',
		"1101": 'D',
		"1110": 'E',
		"1111": 'F'}
	hex = ""
	for i in range(0, len(s), 4):
		ch = ""
		ch = ch + s[i]
		ch = ch + s[i + 1]
		ch = ch + s[i + 2]
		ch = ch + s[i + 3]
		hex = hex + mp[ch]

	return hex

# Binary to decimal conversion

def bin2dec(binary):

	binary1 = binary
	decimal, i, n = 0, 0, 0
	while(binary != 0):
		dec = binary % 10
		decimal = decimal + dec * pow(2, i)
		binary = binary//10
		i += 1
	return decimal

# Decimal to binary conversion


def dec2bin(num):
	res = bin(num).replace("0b", "")
	if(len(res) % 4 != 0):
		div = len(res) / 4
		div = int(div)
		counter = (4 * (div + 1)) - len(res)
		for i in range(0, counter):
			res = '0' + res
	return res

# Permute function to rearrange the bits


def permute(k, arr, n):
	permutation = ""
	for i in range(0, n):
		permutation = permutation + k[arr[i] - 1]
	return permutation

# shifting the bits towards left by nth shifts


def shift_left(k, nth_shifts):
	s = ""
	for i in range(nth_shifts):
		for j in range(1, len(k)):
			s = s + k[j]
		s = s + k[0]
		k = s
		s = ""
	return k

# calculating xow of two strings of binary number a and b

def xor(a, b):
	ans = ""
	for i in range(len(a)):
		if a[i] == b[i]:
			ans = ans + "0"
		else:
			ans = ans + "1"
	return ans

In [4]:
def sbox(sixbits):
	return sixbits[:len(sixbits) - (len(sixbits)-salidaSbox)]

In [10]:
# Table of Position of 64 bits at initial level: Initial Permutation Table
initial_perm = generatePermTab(bloquept)
initial_perm

[1, 8, 5, 3, 2, 6, 7, 4]

In [ ]:
# Expansion D-box Table
exp_d = generateExpTab(mediobloquept,tamaniosubllave)
exp_d

In [ ]:
# Straight Permutation Table
per = generatePermTab(mediobloquept)
per

In [ ]:
# Final Permutation Table
final_perm = generatePermTab(bloquept)
final_perm

In [ ]:
def encrypt(pt, rkb):
	pt = hex2bin(pt)

	# Initial Permutation
	pt = permute(pt, initial_perm, bloquept)
	#print("After initial permutation", bin2hex(pt))

	# Splitting
	left = pt[0:mediobloquept]
	right = pt[mediobloquept:bloquept]
	for i in range(0, 16):
		# Expansion D-box: Expanding the 32 bits data into 48 bits
		right_expanded = permute(right, exp_d, tamaniosubllave)

		# XOR RoundKey[i] and right_expanded
		xor_x = xor(right_expanded, rkb[i])
		# S-boxex: substituting the value from s-box table by calculating row and column
		sbox_str = ""
		for j in range(0, cantSbox):
			sixbits=xor_x[j*6:(j*6)+6]
			sbox_str = sbox_str + sbox(sixbits)
		
		# Straight D-box: After substituting rearranging the bits
		sbox_str = permute(sbox_str, per, mediobloquept)

		# XOR left and sbox_str
		result = xor(left, sbox_str)
		left = result

		# Swapper
		if(i != 15):
			left, right = right, left
		print("Round ", i + 1, " ", left,
			" ", right, " ", rkb[i])

	# Combination
	combine = left + right

	# Final permutation: final rearranging of bits to get cipher text
	cipher_text = permute(combine, final_perm, bloquept)
	return cipher_text

In [ ]:
entrada = "123RADWAASDADKSA"
key = "AABB09182736CCDD"

pt = pt2hex(entrada)
print(pt)

In [ ]:
# Key generation
# --hex to binary
key = hex2bin(key)

# Number of bit shifts
shift_table = [1, 1, 2, 2,
			2, 2, 2, 2,
			1, 2, 2, 2,
			2, 2, 2, 1]

# Key- Compression Table : Compression of key from 56 bits to 48 bits
key_comp = generatePermTab(tamaniosubllave)

# Splitting
left = key[0:int(tamaniollave/2)] 
right = key[int(tamaniollave/2):tamaniollave] 

rkb = [] # rkb for RoundKeys in binary
rk = [] # rk for RoundKeys in hexadecimal
for i in range(0, 16):
	# Shifting the bits by nth shifts by checking from shift table
	left = shift_left(left, shift_table[i])
	right = shift_left(right, shift_table[i])

	# Combination of left and right string
	combine_str = left + right

	# Compression of key from 56 to 48 bits
	round_key = permute(combine_str, key_comp, tamaniosubllave)
	rkb.append(round_key)

In [ ]:
print("Encryption")
cipher_text = bin2hex(encrypt(pt, rkb))
print("Cipher Text : ", cipher_text)